In [ ]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities.
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

# Properly install detectron2. (Please do not install twice in both ways)
# !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
fatal: destination path 'detectron2' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#Check versions of installed libraries
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0
torch:  1.12 ; cuda:  cu113
detectron2: 0.6


In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

##Single Image

In [ ]:
# Upload file to colab, copy path here 
im = cv2.imread("/content/scene00841.jpg")
cv2_imshow(im)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# Use model from detectron2's model zoo.(or https://dl.fbaipublicfiles...) 
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)
outputs = predictor(im)

In [ ]:
print(outputs["instances"].pred_classes)
print(outputs["instances"].pred_boxes)

tensor([ 0, 56,  0, 66, 56, 73, 63, 71, 73, 62, 73, 73, 73, 73, 56, 73, 73, 73,
        60, 73, 76, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 56, 62, 73, 73,
        73, 73, 73, 73, 73, 71], device='cuda:0')
Boxes(tensor([[9.4234e+01, 5.1815e+02, 4.4621e+02, 9.8581e+02],
        [1.1289e+03, 6.6835e+02, 1.4030e+03, 1.0857e+03],
        [5.9006e+02, 4.9982e+02, 1.1356e+03, 1.1580e+03],
        [1.3098e+03, 6.7940e+02, 1.4789e+03, 7.3471e+02],
        [7.4101e+02, 9.1258e+02, 9.8083e+02, 1.2909e+03],
        [1.3536e+03, 2.9594e+02, 1.4100e+03, 3.7989e+02],
        [0.0000e+00, 6.7509e+02, 5.4512e+01, 7.6820e+02],
        [4.9624e+02, 5.9561e+02, 6.6108e+02, 6.5984e+02],
        [1.3100e+03, 3.2498e+02, 1.3667e+03, 3.9269e+02],
        [1.4201e+03, 5.0232e+02, 1.5672e+03, 6.6047e+02],
        [1.2752e+03, 3.4989e+02, 1.3051e+03, 4.1278e+02],
        [1.3403e+03, 3.1098e+02, 1.3823e+03, 3.8307e+02],
        [1.6127e+03, 1.6809e+02, 1.6540e+03, 2.9241e+02],
        [1.5865e+03, 1.7184e+0

In [ ]:
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(out.get_image()[:, :, ::-1])

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Inference with a keypoint detection model
# im = cv2.imread("/content/scene00331.jpg")
cfg = get_cfg()   # get a fresh new config
cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7  # set threshold for this model
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)
outputs = predictor(im)
v = Visualizer(im[:,:,::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(out.get_image()[:, :, ::-1])

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Inference with a panoptic segmentation model
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml")
predictor = DefaultPredictor(cfg)
panoptic_seg, segments_info = predictor(im)["panoptic_seg"]
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_panoptic_seg_predictions(panoptic_seg.to("cpu"), segments_info)
cv2_imshow(out.get_image()[:, :, ::-1])

Output hidden; open in https://colab.research.google.com to view.

## Video stream 

In [ ]:
def keypoints(image):
    cfg = get_cfg()   # get a fresh new config
    cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"))
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7  # set threshold for this model
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml")
    predictor = DefaultPredictor(cfg)
    outputs = predictor(image)
    v = Visualizer(image[:,:,::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    img = out.get_image()[:, :, ::-1]
    # print(img.shape)
    # cv2_imshow(out.get_image()[:, :, ::-1])
    return img

#vidcap object
cap = cv2.VideoCapture('/content/0161ClipShort.mp4')
if (cap.isOpened()== False): 
  print("Error opening video stream or file")

# Default resolutions of the frame are obtained.The default resolutions are system dependent.
# We convert the resolutions from float to integer.
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# if this is not set correctly causes an error, these values were obtained from the o/p of the network
frame_widthp =int(2304)
frame_heightp =int(1728)
#fourcc code for mp4
fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
framerate = 5
# Define the codec and create VideoWriter object.The output is stored in 'output.mp4' file.
outr = cv2.VideoWriter('outputr.mp4',fourcc, framerate, (frame_width,frame_height))
outp = cv2.VideoWriter('outputp.mp4',fourcc, framerate, (frame_widthp,frame_heightp))

# Read until end of video, running model on every 60/framerateth frame(to reduce fps from 60 to framerate)
i = 0
#issue is not having two video writing streams open, its probbaly somethign to do with the codec
while(cap.isOpened()):
  # Capture frame-by-frame
  ret, frame = cap.read()
  if ret == True:
    i+=1
    if i == (60/framerate):
        # cv2_imshow(frame)
        outr.write(frame)
        processed_frame = keypoints(frame)
        cv2_imshow(processed_frame)
        outp.write(processed_frame)
        i=0

    # Press Q on keyboard to  exit
    if cv2.waitKey(25) & 0xFF == ord('q'):
      break
  
  else: 
    break
 
# When done release the video cap and vi write objs
cap.release()
outr.release()
outp.release()

# close frames
cv2.destroyAllWindows()  

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Download the results
from google.colab import files
files.download('outputr.mp4')
files.download('outputp.mp4')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>